<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [2]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [5]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])<= 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

3921
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")

results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  #print(k,v)
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [7]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

3486
173


In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

print (len(results_cleaned2))
vocabulary =  set()


def process_tournament(id):
  task3 = []
  words = []
  for v in results_cleaned2[id]:
    for p in range(len(v[0])): #answers
      for k in v[2]: # players
        quest = str(id)+'_'+str(p)
        player = int(k)
        task3.append((quest, player, v[0][p]) )
        words.append(quest)
        words.append(player)

  if len(words)==0:
    return []
  
  words = set(words)
  word_to_idx = dict([(y,x) for x,y in enumerate(words)])
  idx_to_word = dict([(x,y) for x,y in enumerate(words)])
  
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = task3[i][2]

  logistic = LogisticRegression(fit_intercept = False ) #penalty = 'none', 
  logistic.fit(X, Y)

  result = []
  for s in range(len(logistic.coef_[0])):
    result.append((idx_to_word[s],logistic.coef_[0][s]))
  gc.collect()
  return result

#print(process_tournament(1521))
#process_tournament(6349)

4339


Создадим словарь из всех вопросов и игроков, их можно отличать по наличию символа '_' в названии.

In [9]:
#del all_words
all_words = []

for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

357955


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [0]:
import tqdm
import gc
all_results = {}
for s in all_words:
  all_results[s] = [0,0]

print(all_results)

i=0
for tourn in tqdm.tqdm(train_id):
  i=i+1
  print(tourn)
  result = process_tournament(tourn)
  #for s in result:
  #  all_results[s[0]][1] +=1
  #  all_results[s[0]][0] +=s[1]
  gc.collect()

  0%|          | 0/3486 [00:00<?, ?it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|          | 1/3486 [00:00<46:47,  1.24it/s]

76


  0%|          | 2/3486 [00:02<55:24,  1.05it/s]

141


  0%|          | 3/3486 [00:03<57:07,  1.02it/s]

...

1519


  0%|          | 17/3486 [00:31<3:33:51,  3.70s/it]

1521


  1%|          | 18/3486 [00:40<4:53:13,  5.07s/it]

1526


In [0]:
#print(results_cleaned2[314])
#print(results_cleaned2[1518])

(314, None, 0, [])


In [0]:
ranking = [(k,sum(v)/len(v),len(v)) for k,v in all_results.items() if len(v)>0]
print(ranking)

ranking.sort(key = lambda x: -x[1])

print(len(ranking))

[(144, 0.8115716822453486, 3921), (439, 0.8115716822453486, 3921), (606, 0.3096916142463704, 3921), (668, 1.2453003007722203, 3921), (707, 1.81281595304428, 3921), ('22_61', -0.8014541582410943, 3921), (1055, -0.024618723036230465, 3921), (1145, 0.8115716822453486, 3921), (1560, 2.021934955245346, 3921), (1603, 1.3353444131489387, 3921), (2005, 1.1565697901447998, 3921), (2454, 1.1565697901447998, 3921), (2626, 0.3096916142463704, 3921), (2694, 1.81281595304428, 3921), ('22_89', 2.3396746890502476, 3921), (2935, 2.021934955245346, 3921), (3207, 1.5201563477227515, 3921), (3270, 2.021934955245346, 3921), (3498, -1.2127987362172403, 3921), (3645, 1.5201563477227515, 3921), (4060, 1.2453003007722203, 3921), (4551, 1.1565697901447998, 3921), (4576, 0.9823792319043849, 3921), ('22_46', -20.77005188280133, 3921), (4878, 2.021934955245346, 3921), (5526, 1.7128253338578534, 3921), (5935, 1.3353444131489387, 3921), (6020, 0.4761072327512964, 3921), (6212, 1.5201563477227515, 3921), (6625, 0.811

In [0]:
def isquestion():
  if 

24